# Assignment 7: CNNs & RNNs

In [ ]:
# enter your name and UFL email address
name = 'enter your name'
email = 'enter your email'

In [ ]:
if name == 'enter your name' or email == 'enter your email':
    assert False, 'Enter your name & email first!'
else:
    print('Assignment 7 -- name: {}, email: {}\n'.format(name, email))
    
    # Load packages we need
    import sys
    import os
    import time

    import numpy as np
    import sklearn
    
    # we'll use tensorflow and keras for neural networks
    import tensorflow as tf
    import tensorflow.keras as keras
    
    # import layers we may use
    from tensorflow.keras.layers import Input, Flatten, Dense, Conv2D, MaxPooling2D, Dropout

    # import callbacks we may use
    from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
    
    # Load the TensorBoard notebook extension
    #%load_ext tensorboard

    from matplotlib import pyplot as plt
    plt.rcParams.update({'font.size': 16})

    # Let's check our software versions
    print('### Python version: ' + __import__('sys').version)
    print('### NumPy version: ' + np.__version__)
    print('### Scikit-learn version: ' + sklearn.__version__)
    print('### Tensorflow version: ' + tf.__version__)
    print('### TF Keras version: ' + keras.__version__)
    print('------------')


    # load our packages / code
    sys.path.insert(1, '../common/')
    import utils
    import plots

In [ ]:
# global parameters to control behavior of the pre-processing, ML, analysis, etc.
seed = 42

# deterministic seed for reproducibility
np.random.seed(seed)
tf.random.set_seed(seed)

prop_vec = [24, 2, 2]

## [Task 1] (20 points) Loading and Processing CIFAR-10

### [Task 1a] (20 points) Complete the implementation of load_preprocess_cifar10(). Make sure you correctly implement all of the cases.

In [ ]:
from tensorflow.keras.datasets import cifar10

# refer to: https://www.tensorflow.org/api_docs/python/tf/keras/datasets/cifar10/load_data
# and to https://www.cs.toronto.edu/~kriz/cifar.html
def load_preprocess_cifar10(onehot=True, minmax_normalize=True):
    
    labels = np.array(['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'])
    
    ### Load and preprocess the cifar10 data, then split it into train, test, validation
    ### The shapes of train_x, test_x, val_x should be: (50000, 32, 32, 3), (5000, 32, 32, 3), (5000, 32, 32, 3)
    ### If onehot=True you need to one hot encode the labels (y vector)
    ### If minmax_normalize=True you need to minmax normalize the pixel values to be in the range [0,1]
    ###* put your code here (~10-20 lines) *###

    
    
    return train_x, train_y, test_x, test_y, val_x, val_y, labels

In [ ]:
# do some sanity checks
train_x, train_y, test_x, test_y, val_x, val_y, labels = load_preprocess_cifar10(onehot=False, minmax_normalize=False)
assert train_x.shape[0] == train_y.shape[0] and test_x.shape[0] == test_y.shape[0] and val_x.shape[0] == val_y.shape[0]
assert np.amax(train_x) >= 255 and np.amax(test_x) >= 255 and np.amax(val_x) >= 255
assert train_y.shape == (train_y.shape[0],) or train_y.shape == (train_y.shape[0],1)

train_x, train_y, test_x, test_y, val_x, val_y, labels = load_preprocess_cifar10(onehot=True, minmax_normalize=False)
assert np.amax(train_x) >= 255 and np.amax(test_x) >= 255 and np.amax(val_x) >= 255
assert train_y.shape == (train_y.shape[0],10) and train_y.shape[1] == test_y.shape[1]

In [ ]:
# actually load the data
train_x, train_y, test_x, test_y, val_x, val_y, labels = load_preprocess_cifar10()
assert np.amax(train_x) <= 1 and np.amax(test_x) <= 1 and np.amax(val_x) <= 1
assert np.amax(train_x) >= 0 and np.amax(test_x) >= 0 and np.amax(val_x) >= 0

assert labels.shape[0] == 10 and labels.shape[0] == train_y.shape[1]

## [Task 2] (30 points) Training a CNN for Cifar-10

#### We will use the following architecture
- Conv layer with 32 filters, (3,3) filter size, stride of 1, padding 'same'
- Conv layer with 32 filters, (3,3) filter size, stride of 1, padding 'same'
- Max pooling layer (2,2)
- Dropout with rate 25%
- Conv layer with 64 filters, (3,3) filter size, stride of 1, padding 'same'
- Conv layer with 64 filters, (3,3) filter size, stride of 1, padding 'same'
- Max pooling layer (2,2)
- Dropout with rate 25%
- Conv layer with 128 filters, (3,3) filter size, stride of 1, padding 'same'
- Conv layer with 128 filters, (3,3) filter size, stride of 1, padding 'same'
- Max pooling layer (2,2)
- Dropout with rate 25%
- Flatten
- FC with 128 units
- Dropout with rate 25%
- FC with 64 units
- Dropout with rate 25%
- (Output layer) FC with 10 units

#### For all layers (if applicable) except the output layer you should use:
- ReLU as activation function
- He uniform weight initialization strategy
- L2 regularization with regularization constant set to 0.001

#### For the output layer you should select a suitable activation function that is consistent with the task and loss function you use. Use Adam for the optimizer with learning rate 0.002.

### [Task 2a] (20 points) Implement create_compile_cnn() according to the architecture specified above.

In [ ]:
def create_compile_cnn(input_shape=[32, 32, 3], num_outputs=10, verbose=False):
    
    model = keras.models.Sequential(name='CIFAR-10--CNN')
    
    ### Don't forget to compile the model and print the summary if verbose=True
    ###* put your code here (~20 lines) *###
    
    
    
    
    
    return model

In [ ]:
_ = create_compile_cnn(verbose=True)

### [Task 2b] (10 points) Train the model. Fill in the implementation below.

In [ ]:
cnn_model_fp = './cifar10-cnn.h5'

# If the model file exists, load it. Otherwise train it and save the model.
# Note: if you need to retrain the model, simply delete the h5 file.
if os.path.exists(cnn_model_fp):
    model = keras.models.load_model(cnn_model_fp)
else:
    model = create_compile_cnn(verbose=False)
    
    # train the model using model.fit() for at least 3 epochs and your chosen batch_size
    # you can set any callback you want on it, including checkpoint, early stopping, etc.
    ###* put your code here (~3-5 lines) *###
    
    
    
    # save the model
    model.save(cnn_model_fp)

In [ ]:
# let's evaluate the model on the test data
loss, acc = model.evaluate(test_x, test_y, verbose=0)
print('[Model] Test accuracy: {:.2f}%'.format(100*acc))

## [Task 3] (15 points) Processing Sequence Data

### [Task 3a] (15 points) Fill in the implementation of load_preprocess_imdb()

In [ ]:
from tensorflow.keras.datasets import imdb

# the size of the vocabulary we'll use
vocab_size = 12000
maxlen = 150

# refer to: https://www.tensorflow.org/api_docs/python/tf/keras/datasets/cifar10/load_data
# and to https://www.cs.toronto.edu/~kriz/cifar.html
def load_preprocess_imdb(num_words=vocab_size, prop_vec=prop_vec, maxlen=maxlen, vectorize=False):
    
    np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)
    
    train, testval = imdb.load_data(num_words=num_words, maxlen=maxlen, oov_char=0)
    
    np.warnings.filterwarnings('default', category=np.VisibleDeprecationWarning)    
    
    ### Process the data 
    ### Merge train and testval, but then split again into train, test, val sets (according to prop_vec). You can use utils.train_test_val_split().)
    ### - If vectorize=True, then you must encode the features of each example into vectors of vocab_size entries 
    ### such that entry i contains the number of time word i appeared in the sequence
    ### - If vectorize=False, then you must encode the features of each examples as a sequence of size maxlen (represented as a np.array()). 
    ### Make sure to pad sequences with 0 as appropriate.
    ###* put your code here (~10-15 lines) *###
    
    
    
    
    
    return train_x, train_y, test_x, test_y, val_x, val_y

In [ ]:
# sanity checks
train_x, train_y, test_x, test_y, val_x, val_y = load_preprocess_imdb(vectorize=False)
assert train_x.shape == (16281, maxlen) and train_y.shape == (train_x.shape[0],)

train_x, train_y, test_x, test_y, val_x, val_y = load_preprocess_imdb(vectorize=True)
assert train_x.shape == (16281, vocab_size) and train_y.shape == (train_x.shape[0],)

## [Task 4] (35 points) RNN for Sentiment Analysis

### [Task 4a] (35 points) Complete the code below to define an RNN architecture for sentiment analysis. The goal is to predict the sentiment of IMDB reviews. You can use any architecture you want, but a good place to start would be to use an embedding layer followed by some recurrent layers (e.g., LSTM, GRU, etc.). Keep the number of parameters of the model below 2m.

In [ ]:
def create_compile_rnn(input_shape=[None], vocab_size=vocab_size, embedding_size=128, num_outputs=1, verbose=False):
    
    model = keras.models.Sequential(name='imdb-RNN')
    
    ### Don't forget to compile the model and print the summary if verbose=True
    ### Use binary_crossentropy as loss function.    
    ###* put your code here (~15-20 lines) *###

    
    
    return model

In [ ]:
model = create_compile_rnn(verbose=True)

In [ ]:
# let's load the data
train_x, train_y, test_x, test_y, val_x, val_y = load_preprocess_imdb(vectorize=False)


early_stop_cb = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# feel free to tweak the batch size, number of epochs and callbacks.
max_epochs = 3
batch_size = 32

hist = model.fit(train_x, train_y, epochs=max_epochs, batch_size=batch_size, validation_data=(val_x, val_y), callbacks=[early_stop_cb])

In [ ]:
# let's evaluate the model on the test data
loss, acc = model.evaluate(test_x, test_y, verbose=0)
print('[Model] Test accuracy: {:.2f}%'.format(100*acc))

## [CIS6930 Additional Task -- Task 5] (25 points): DNN for Sentiment Analysis

### In the previous task, we use an RNN for sentiment analysis. In this task you will use a neural network without any recurrent layers for the same task as a comparison.

### We'll use the data in vectorized form for this.

### [Task 5a] (20 points) Complete the code below to define an architecture of your choice *without* any recurrent layers. The goal is to get the best model with the fewest number of parameters. Keep the number of parameters of the model below 2m and ideally similar to the model of Task 4.

In [ ]:
def create_compile_dnn(input_shape=[vocab_size], num_outputs=1, verbose=False):
    
    model = keras.models.Sequential(name='imdb-DNN')
    
    ### Don't forget to compile the model and print the summary if verbose=True
    ###* put your code here (~10 lines) *###

    
    return model

In [ ]:
model = create_compile_dnn(verbose=True)

In [ ]:
# Let's load the data in vectorized form
train_x, train_y, test_x, test_y, val_x, val_y = load_preprocess_imdb(vectorize=True)


early_stop_cb = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# feel free to tweak the batch size, number of epochs and callbacks.
max_epochs = 50
batch_size = 100

hist = model.fit(train_x, train_y, epochs=max_epochs, batch_size=batch_size,validation_data=(val_x, val_y), callbacks=[early_stop_cb])

In [ ]:
# let's evaluate the model on the test data
loss, acc = model.evaluate(test_x, test_y, verbose=0)
print('[Model] Test accuracy: {:.2f}%'.format(100*acc))

### [Task 5b] (5 points) Compare this model to the model of Task 4. What do you conclude?

In [ ]:
###* put your answer here *###
#
# 
#